# Indonesian E-commerce Review Summarization with Mistral-7B-Instruct

This notebook demonstrates how to use Mistral-7B-Instruct for abstractive summarization of Indonesian e-commerce reviews.

## Setup

First, install the required dependencies if you haven't already:

```bash
pip install -r requirements.txt
```

In [ ]:
# Import required libraries
import sys
sys.path.append('../src')

from indo_ecommerce_review_summarization.preprocessing import clean_text, normalize_text
from indo_ecommerce_review_summarization.models import create_summarization_prompt, load_model
from indo_ecommerce_review_summarization.evaluation import calculate_rouge

import torch
print(f"PyTorch version: {torch.__version__}")
print(f"CUDA available: {torch.cuda.is_available()}")

## Load Model

Load Mistral-7B-Instruct model. You can use quantization to reduce memory usage.

In [ ]:
# Load model - use 4-bit quantization to save memory
model = load_model(
    model_name="mistralai/Mistral-7B-Instruct-v0.2",
    model_type="huggingface",
    load_in_4bit=True,
    torch_dtype=torch.float16
)

print("Model loaded successfully!")

## Example 1: Single Review Summarization

In [ ]:
# Example Indonesian e-commerce review
review = """
Barang udah sampai dgn selamat. Packaging rapi bgt, bubble wrap tebal jd gak khawatir pecah. 
Kualitas produk oke, sesuai deskripsi. Harga agak mahal tp worth it sih. 
Pengiriman cepet bgt cuma 2 hari sampe. Seller responsif, fast respon banget. 
Puas bgt sama pembelian kali ini. Recommended seller! 👍
"""

# Preprocess the review
cleaned_review = clean_text(review)
print("Cleaned review:")
print(cleaned_review)
print("\n" + "="*80 + "\n")

# Create prompt
prompt = create_summarization_prompt(
    reviews=[cleaned_review],
    model_type="mistral",
    max_length=50
)

print("Prompt:")
print(prompt)
print("\n" + "="*80 + "\n")

# Generate summary
summary = model.generate(
    prompt,
    max_new_tokens=128,
    temperature=0.7,
    top_p=0.9
)

print("Generated Summary:")
print(summary)

## Example 2: Multiple Reviews Summarization

In [ ]:
# Multiple reviews
reviews = [
    "Produk bagus banget, sesuai ekspektasi. Pengiriman cepat.",
    "Kualitas oke, tapi pengiriman agak lama. Overall puas sih.",
    "Barang mantap! Seller ramah dan fast respon. Recommended!",
    "Harga sebanding dengan kualitas. Packing rapi. Good job!"
]

# Create prompt for multiple reviews
prompt = create_summarization_prompt(
    reviews=reviews,
    model_type="mistral"
)

# Generate summary
summary = model.generate(
    prompt,
    max_new_tokens=150,
    temperature=0.7
)

print("Summary of multiple reviews:")
print(summary)

## Example 3: Evaluation with ROUGE

In [ ]:
# Example: evaluate against reference summary
reference_summary = "Produk berkualitas dengan pengiriman cepat dan seller yang responsif."
predicted_summary = summary  # Use the generated summary from above

# Calculate ROUGE scores
scores = calculate_rouge(
    predictions=predicted_summary,
    references=reference_summary
)

print("ROUGE Scores:")
for metric, values in scores.items():
    print(f"\n{metric.upper()}:")
    for key, value in values.items():
        print(f"  {key}: {value:.4f}")

## Example 4: Batch Processing

In [ ]:
# Multiple reviews to summarize
reviews_list = [
    ["Barang bagus, pengiriman cepat, seller ramah."],
    ["Produk sesuai deskripsi, packing aman."],
    ["Kualitas oke, harga terjangkau. Recommended!"]
]

# Create prompts for all reviews
prompts = [create_summarization_prompt(reviews, model_type="mistral") for reviews in reviews_list]

# Generate summaries in batch
summaries = model.batch_generate(
    prompts,
    max_new_tokens=100,
    temperature=0.7,
    batch_size=2
)

# Display results
for i, (review, summary) in enumerate(zip(reviews_list, summaries), 1):
    print(f"\nReview {i}: {review[0]}")
    print(f"Summary: {summary}")
    print("-" * 80)

## Conclusion

This notebook demonstrated:
1. Loading and using Mistral-7B-Instruct for Indonesian review summarization
2. Text preprocessing for Indonesian e-commerce reviews
3. Single and multiple review summarization
4. Evaluation using ROUGE metrics
5. Batch processing for efficiency

## Next Steps

- Experiment with different prompt templates
- Try aspect-based summarization
- Fine-tune the model on your specific dataset
- Compare with other models (LLaMA, GPT, etc.)